# Continuous Control-Reacher

---

In this notebook, you will be guided through the steps to create an agent, load the train weight for the actor, and see the performance of a trained agent. This notebook only works for MacOSX, for other operation systems please change the name of envirnoment accordingly.

Remember to change the kernel to 'drlnd', which can be set up following the [here](https://github.com/udacity/deep-reinforcement-learning#dependencies)

## 1. Before we start

Before we get started, please make sure that all necessary files are in the same folder as this notebook. Please also make sure that they are not in other sub-folders. The requirements are:
    * actor_checkpoint.pth, which is the trained weights for actor;s network
    * infrastructures.py
    * agents.py
    * the environment file, please make sure the name is 'Reacher'
    

## 2. Start the Environment

In [1]:
from unityagents import UnityEnvironment

In [3]:
# start environment, you might need to change the name
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [15]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('Size of each state:', state_size)

Number of agents: 20
Size of each action: 4
Size of each state: 33


In [5]:
import numpy as np
import torch
from collections import namedtuple, deque

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## 3. Load .py files

For the following two blocks, you will have to run each of them twice. The first time will load the .py files into this notebook. The second time will actually execute the code in this notebook.

In [7]:
# %load infrastructures.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple, deque
import numpy as np
import random
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using device {}'.format(device))

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        
        experiences = random.sample(self.memory, k=self.batch_size)
        
        #Extract information from memory unit and return
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)    
    
class Actor(nn.Module):
    
    def __init__(self, state_size, action_size, hidden=[400, 300]):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.fc3 = nn.Linear(hidden[1], action_size)
        self.bn1 = nn.BatchNorm1d(hidden[0])
        self.bn2 = nn.BatchNorm1d(hidden[1])
        self.initialize()
        
    def forward(self, x):
        x = self.bn1(F.relu(self.fc1(x)))
        x = self.bn2(F.relu(self.fc2(x)))
        x = F.tanh(self.fc3(x))
        return x
    
    def initialize(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc1.bias.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc2.bias.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
        self.fc3.bias.data.uniform_(-3e-3, 3e-3)
        
class Critic1(nn.Module):
    
    def __init__(self, state_size, action_size, hidden=[400, 300]):
        super(Critic1, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0]+action_size, hidden[1])
        self.fc3 = nn.Linear(hidden[1], 1)
        self.bn1 = nn.BatchNorm1d(hidden[0])
        self.initialize()
    
    def forward(self, state, action):
        x = F.relu(self.fc1(state))
        x = torch.cat([x, action], dim=1)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def initialize(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc1.bias.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc2.bias.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-4, 3e-4)
        self.fc3.bias.data.uniform_(-3e-4, 3e-4)
        
class Critic2(nn.Module):
    
    def __init__(self, state_size, action_size, hidden=[400, 300]):
        super(Critic2, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.fc3 = nn.Linear(hidden[1]+action_size, 1)
        self.bn1 = nn.BatchNorm1d(hidden[0])
        self.bn2 = nn.BatchNorm1d(hidden[1])
        self.initialize()
    
    def forward(self, state, action):
        x = self.bn1(F.relu(self.fc1(state)))
        x = self.bn2(F.relu(self.fc2(x)))
        x = torch.cat([x, action], dim=1)
        x = self.fc3(x)
        return x
    
    def initialize(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc1.bias.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc2.bias.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-4, 3e-4)
        self.fc3.bias.data.uniform_(-3e-4, 3e-4)
        
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, agent_num, action_size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones((agent_num, action_size))
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()
        self.shape = (agent_num, action_size)

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(self.shape[0]*self.shape[1])]).reshape(self.shape)
        self.state = x + dx
        return self.state

Using device cpu


In [9]:
# %load agents.py
import torch
import torch.nn
import torch.nn.functional as F
from infrastructures import Actor, Critic1, Critic2, ReplayBuffer, OUNoise, device
import torch.optim as optim
import numpy as np
import random
from collections import deque

class DDPG_Agent:
    
    def __init__(self, env, critic, lr1=0.0001, lr2=0.001, tau=0.001, speed1=1, speed2=1,\
                 step=1, learning_time=1, batch_size=64):
        
        #Initialize environment
        brain_name = env.brain_names[0]
        brain = env.brains[brain_name]
        env_info = env.reset(train_mode=True)[brain_name]
        num_agents = len(env_info.agents)
        action_size = brain.vector_action_space_size
        states = env_info.vector_observations
        state_size = states.shape[1]
        self.env = env
        
        #Initialize some hyper parameters of agent
        self.lr1 = lr1
        self.lr2 = lr2
        self.tau = tau
        self.speed1 = speed1
        self.speed2 = speed2
        self.learning_time = learning_time
        self.state_size = state_size
        self.action_size = action_size
        self.num_agents = num_agents
        self.batch_size = batch_size
        self.gamma = 0.99
        self.step = step
        
        #Initialize agent (networks, replyabuffer and noise)
        self.actor_local = Actor(self.state_size, self.action_size).to(device)
        self.actor_target = Actor(self.state_size, self.action_size).to(device)
        if critic==1:
            self.critic_local = Critic1(self.state_size, self.action_size).to(device)
            self.critic_target = Critic1(self.state_size, self.action_size).to(device)
        else:
            self.critic_local = Critic2(self.state_size, self.action_size).to(device)
            self.critic_target = Critic2(self.state_size, self.action_size).to(device)
        self.soft_update(self.actor_local, self.actor_target, 1)
        self.soft_update(self.critic_local, self.critic_target, 1)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=self.lr1)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=self.lr2)
        self.memory = ReplayBuffer(self.action_size, buffer_size=int(1e6), batch_size=self.batch_size,\
                                   seed=random.randint(1, self.batch_size))
        self.noise = OUNoise(agent_num=self.num_agents, action_size=self.action_size, seed=random.randint(1, self.batch_size))
        
    def act(self, state, i):
        state = torch.tensor(state, dtype=torch.float).to(device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).detach().cpu().numpy()
        self.actor_local.train()
        noise = self.noise.sample()
        action += noise/np.sqrt(i)
        action = np.clip(action, -1, 1)
        return action
        
    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            
    def learn(self):
        
        experiences = self.memory.sample()
        states, actions, scores, next_states, dones = experiences
        
        expected_rewards = scores + (1-dones)*self.gamma*self.critic_target(next_states, self.actor_target(next_states))
        
        for _ in range(self.speed1):
            observed_rewards = self.critic_local(states, actions)
            L = F.mse_loss(expected_rewards, observed_rewards)
            self.critic_optimizer.zero_grad()
            L.backward()
            torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
            self.critic_optimizer.step()
            del L
        
        for _ in range(self.speed2):
            L = -self.critic_local(states, self.actor_local(states)).mean()
            self.actor_optimizer.zero_grad()
            L.backward()
            self.actor_optimizer.step()
            del L
        
        self.soft_update(self.actor_local, self.actor_target, self.tau)
        self.soft_update(self.critic_local, self.critic_target, self.tau)
        
            
    def train(self, n_episodes):
        rewards = []
        brain_name = self.env.brain_names[0]
        score_window = deque(maxlen=100)
        
        for i in range(1, n_episodes+1):
            episodic_reward = np.zeros(self.num_agents)
            env_info = self.env.reset(train_mode=True)[brain_name]
            states = env_info.vector_observations
            actions = self.act(states, i)
            t=0
            
            while True:
                env_info = self.env.step(actions)[brain_name]
                next_states = env_info.vector_observations
                dones = env_info.local_done
                scores = env_info.rewards
                episodic_reward += np.array(scores)
                for state, action, score, next_state, done in zip(states, actions, scores, next_states, dones):
                    self.memory.add(state, action, score, next_state, done)
                t += 1
                
                if len(self.memory.memory)>self.batch_size:
                    if t % self.step == 0:
                        for _ in range(self.learning_time):
                            self.learn()
                
                if any(dones):
                    break
                            
                states = next_states
                actions = self.act(states, i)
            score_window.append(np.mean(episodic_reward))
            rewards.append(episodic_reward)
            
            print('\rEpisode {}. Total score for this episode: {:.4f}, average score {:.4f}'.format(i, np.mean(episodic_reward),np.mean(score_window)),end='')
            if i % 100 == 0:
                print('')
        
        np.save('./offline/offline_rewards.npy',np.array(rewards))
        self.actor_local.cpu()
        self.critic_local.cpu()
        torch.save(self.actor_local.state_dict(),'./offline/actor_checkpoint.pth')
        torch.save(self.critic_local.state_dict(),'./offline/critic_checkpoint.pth')
            

Using device cpu


## 4. See the performance

In [18]:
agent = DDPG_Agent(env=env, critic=1) 

In [20]:
agent.actor_local.load_state_dict(torch.load('actor_checkpoint.pth'))

In [21]:
agent.actor_local.eval()

Actor(
  (fc1): Linear(in_features=33, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=300, bias=True)
  (fc3): Linear(in_features=300, out_features=4, bias=True)
  (bn1): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [22]:
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
t = 0                                                  # count how many time steps are there
while True:
    t += 1
    actions = agent.actor_local(torch.tensor(states,dtype=torch.float)).detach().cpu().numpy() # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 37.369499164726584
Total number of steps: 1001


## 5. Close the environment

In [23]:
env.close()